In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Data/raw_data/Transaction.csv')
df.head(2)

,Transaction ID,Customer ID,Product ID,Product Category,Purchase Amount,Purchase Date
0,1,238,45,Sensitive Skin Treatment,25,2024-03-24
1,1,238,35,Lip Care,15,2024-03-24


In [3]:
# Step 1: Create a customer-product matrix (purchase count per product)
customer_product_matrix = df.pivot_table(
    index='Customer ID',
    columns='Product ID',
    values='Purchase Amount',
    aggfunc='count',
    fill_value=0
)

In [4]:
# Step 2: Compute cosine similarity between customers
customer_similarity = cosine_similarity(customer_product_matrix)
similarity_df = pd.DataFrame(customer_similarity, index=customer_product_matrix.index, columns=customer_product_matrix.index)

In [5]:
# Step 3: Generate recommendations
recommendations = {}

for customer_id in customer_product_matrix.index:
    # Find top 5 similar customers (excluding the customer themself)
    similar_customers = similarity_df[customer_id].drop(customer_id).sort_values(ascending=False).head(5).index
    
    # Get products purchased by similar customers
    similar_customers_purchases = customer_product_matrix.loc[similar_customers]
    summed_purchases = similar_customers_purchases.sum(axis=0)
    
    # Remove products already purchased by the customer
    already_purchased = customer_product_matrix.loc[customer_id]
    products_to_recommend = summed_purchases[already_purchased == 0]
    
    # Get top 5 recommended product IDs
    top_5_products = products_to_recommend.sort_values(ascending=False).head(5).index.tolist()
    recommendations[customer_id] = top_5_products

In [6]:
# Convert to DataFrame for display
recommendation_df = pd.DataFrame([
    {'Customer ID': customer, 'Recommended Products': products}
    for customer, products in recommendations.items()
])

In [7]:
recommendation_df.head(10)

,Customer ID,Recommended Products
0,1,"[31, 32, 33, 44, 45]"
1,2,"[34, 1, 31, 32, 33]"
2,3,"[34, 21, 31, 32, 33]"
3,4,"[19, 3, 22, 46, 1]"
4,5,"[22, 2, 31, 32, 33]"
5,6,"[39, 2, 22, 31, 32]"
6,7,"[48, 1, 18, 2, 31]"
7,8,"[3, 36, 1, 45, 48]"
8,9,"[45, 21, 1, 2, 5]"
9,10,"[19, 31, 32, 33, 49]"


In [8]:
# Get past purchases for each customer
customer_purchases = df.groupby('Customer ID')['Product ID'].apply(lambda x: list(sorted(set(x)))).reset_index()

# Merge with recommendations
full_recommendation_df = customer_purchases.merge(recommendation_df, on='Customer ID')

full_recommendation_df.head(10)


,Customer ID,Past Purchases,Recommended Products
0,1,"[1, 2, 3, 4, 5, 18, 19, 20, 21, 22, 34, 35, 36...","[31, 32, 33, 44, 45]"
1,2,"[2, 3, 4, 5, 18, 19, 20, 21, 22, 35, 36, 39, 4...","[34, 1, 31, 32, 33]"
2,3,"[1, 2, 3, 4, 5, 18, 19, 20, 22, 35, 36, 39, 40...","[34, 21, 31, 32, 33]"
3,4,"[2, 4, 5, 20, 21, 34, 35, 36, 39, 40, 47, 48]","[19, 3, 22, 46, 1]"
4,5,"[1, 3, 4, 5, 18, 19, 20, 21, 34, 35, 36, 39, 4...","[22, 2, 31, 32, 33]"
5,6,"[1, 3, 4, 5, 18, 19, 20, 21, 34, 35, 36, 40, 4...","[39, 2, 22, 31, 32]"
6,7,"[3, 4, 5, 19, 20, 21, 22, 34, 35, 36, 39, 40, ...","[48, 1, 18, 2, 31]"
7,8,"[2, 4, 5, 18, 19, 20, 21, 22, 34, 35, 39, 40, ...","[3, 36, 1, 45, 48]"
8,9,"[3, 4, 18, 20, 22, 34, 35, 36, 39, 40, 44, 46,...","[45, 21, 1, 2, 5]"
9,10,"[1, 2, 3, 4, 5, 18, 20, 21, 22, 34, 35, 36, 39...","[19, 31, 32, 33, 49]"


In [9]:
full_recommendation_df.to_csv("Data/results/Product_Recommendation_Collaborative_Filtering.csv", index=False)

# 🧴 Product Recommendation Example — Customer ID: 238

In [11]:
filtered_df = full_recommendation_df[full_recommendation_df['Customer ID'] == 238]
filtered_df

,Customer ID,Past Purchases,Recommended Products
237,238,"[1, 4, 5, 18, 20, 21, 22, 31, 32, 33, 34, 35, ...","[19, 3, 2, 39, 40]"


### 🧾 Customer Purchase History

This customer has previously purchased **17 different skincare products**, including:
- **Cleansers:** Gel Cleansers, Micellar Water, Cleansing Balm  
- **Lip Care:** Lip Balm, Lip Scrub, Lip Mask  
- **Moisturizers:** Gel Moisturizer, Night Cream, Oil-Based Moisturizer, Water-Based Moisturizer  
- **Acne/Pore Care:** Pore Minimizing Serum, Pore Strips  
- **Others:** Calming Serums, Eye Cream, Eye Gel, Soothing Creams, Under-Eye Patches

---

The most similar customers based on cosine similarity of product purchase patterns are:

| Customer ID | Similarity Score |
|:-------------:|:------------------:|
| 359         | 0.894            |
| 252         | 0.846            |
| 204         | 0.846            |
| 304         | 0.838            |
| 320         | 0.831            |


These customers have purchased products in a pattern most similar to Customer 238.

---
### 💡 Top 5 Recommended Products

These recommendations are based how many of the customer's **top N most similar users** have purchased that product.:

| SKU                   | Reason for Recommendation                                             | Score |
|:------------------------:|:----------------------------------------------------------------------:|:--------:|
| **Soothing Creams**     | Common among users with moisturizing preferences                    | 6     |
| **Oil Cleanser**        | Frequently purchased by customers with similar cleansing routines    | 5     |
| **Foam Cleanser**       | Frequently purchased by customers with similar cleansing routines    | 4     |
| **Pimple Patches**      | Popular with customers who share acne treatment habits               | 4     |
| **Acne Spot Treatment** | Purchased by customers with similar skincare patterns                | 4     |



---

### ✅ Recommendation Logic

- These products were not bought by Customer 238.
- But they were frequently bought by the top 5 most similar customers, as determined by cosine similarity of purchase patterns.
- The score represents how many of those similar customers bought the item.

---
